In [22]:
import numpy as np
import operator
import matplotlib.pyplot as plt
%matplotlib inline
import random
from typing import Dict, Tuple, List

In [23]:
class GridWorld:
    def __init__(self):
        self.height = 5
        self.width = 5
        self.actions = ['north', 'south', 'east', 'west']
        
        # special states A and B
        self.special_states = {
            (0, 1): {'next_state': (4, 1), 'reward': 10},  # A → A′
            (0, 3): {'next_state': (2, 3), 'reward': 5}    # B → B′
        }
        self.reset()

    def reset(self) -> Tuple[int,int]:
        # random start
        self.current_state = (random.randint(0, 4), random.randint(0, 4))
        return self.current_state

    def get_available_actions(self) -> List[str]:
        return self.actions

    def step(self, action: str) -> Tuple[Tuple[int,int], float]:
        state = self.current_state

        # special transition?
        if state in self.special_states:
            special = self.special_states[state]
            self.current_state = special['next_state']
            return self.current_state, special['reward']
        
        # normal move
        x, y = state
        if action == 'north':
            new_state = (x-1, y) if x > 0 else state
        elif action == 'south':
            new_state = (x+1, y) if x < self.height-1 else state
        elif action == 'east':
            new_state = (x, y+1) if y < self.width-1 else state
        elif action == 'west':
            new_state = (x, y-1) if y > 0 else state
        else:
            raise ValueError("Invalid action")

        # hit wall?
        reward = -1 if new_state == state else 0
        self.current_state = new_state
        return new_state, reward

    def render(self):
        grid = np.full((self.height, self.width), '.')
        grid[self.current_state] = 'A'
        print(grid)


In [24]:

class QTable:
    def __init__(self, states: List[Tuple[int,int]], actions: List[str]):
        self.q_table = {
            state: {action: 0.0 for action in actions}
            for state in states
        }

    def get_q_value(self, state: Tuple[int,int], action: str) -> float:
        return self.q_table[state][action]

    def get_max_q_value(self, state: Tuple[int,int]) -> float:
        return max(self.q_table[state].values())

    def get_best_action(self, state: Tuple[int,int]) -> str:
        max_q = self.get_max_q_value(state)
        best_actions = [a for a,q in self.q_table[state].items() if q == max_q]
        return random.choice(best_actions)

    def update(self,
               state: Tuple[int,int],
               action: str,
               reward: float,
               next_state: Tuple[int,int],
               alpha: float,
               gamma: float):
        current_q = self.get_q_value(state, action)
        max_next_q = self.get_max_q_value(next_state)
        new_q = current_q + alpha * (reward + gamma * max_next_q - current_q)
        self.q_table[state][action] = new_q


def get_optimal_policy(qtable: QTable) -> Dict[Tuple[int,int], str]:
    policy = {}
    for state in qtable.q_table:
        max_q = qtable.get_max_q_value(state)
        best_actions = [a for a,q in qtable.q_table[state].items() if q == max_q]
        policy[state] = random.choice(best_actions)
    return policy

In [26]:
# ——— Setup ———
env = GridWorld()
states = [(i,j) for i in range(env.height) for j in range(env.width)]
qtable = QTable(states, env.actions)

# ——— Hyperparameters (exact) ———
episodes = 5000
steps    = 5000
alpha    = 0.2
gamma    = 0.9
epsilon  = 0.1

# ——— Initialization printout ———
print("Initializing Gridworld...")
print(f"Grid size: {env.height}x{env.width}")
print("Special_states = {'A': (0,1), 'B': (0,3)}")
print("Next_to_states = {\"A'\": (4,1), \"B'\": (2,3)}")
print("Special_rewards = {'A': 10, 'B': 5}")
print("Starting Q-learning with parameters:")
print(f"  γ = {gamma}")
print(f"  ε = {epsilon}")
print(f"  α = {alpha}")
print(f"  Episodes = {episodes}")
print(f"  Steps = {steps}\n")

# ——— Training ———
for _ in range(episodes):
    state = env.reset()
    for __ in range(steps):
        if random.random() < epsilon:
            action = random.choice(env.actions)
        else:
            action = qtable.get_best_action(state)
        nxt, reward = env.step(action)
        qtable.update(state, action, reward, nxt, alpha, gamma)
        state = nxt

# ——— Evaluation printout ———
print("Evaluating optimal value function and policy...\n")

# Value function
V = np.zeros((env.height, env.width))
for i in range(env.height):
    for j in range(env.width):
        V[i,j] = qtable.get_max_q_value((i,j))

print("Optimal Value Function:")
for i in range(env.height):
    print(" ".join(f"{V[i,j]:5.2f}" for j in range(env.width)))

# Policy names
pi = get_optimal_policy(qtable)
print("\nOptimal Policy:")
for i in range(env.height):
    print(" ".join(pi[(i,j)].ljust(5) for j in range(env.width)))

# Policy arrows
arrow = {'north':'↑','south':'↓','east':'→','west':'←'}
print("\nOptimal Policy (arrows):")
for i in range(env.height):
    print(" ".join(arrow[pi[(i,j)]] for j in range(env.width)))








Initializing Gridworld...
Grid size: 5x5
Special_states = {'A': (0,1), 'B': (0,3)}
Next_to_states = {"A'": (4,1), "B'": (2,3)}
Special_rewards = {'A': 10, 'B': 5}
Starting Q-learning with parameters:
  γ = 0.9
  ε = 0.1
  α = 0.2
  Episodes = 5000
  Steps = 5000

Evaluating optimal value function and policy...

Optimal Value Function:
21.98 24.42 21.98 19.42 17.48
19.78 21.98 19.78 17.80 16.02
17.80 19.78 17.80 16.02 14.42
16.02 17.80 16.02 14.42 12.98
14.42 16.02 14.42 12.98 11.68

Optimal Policy:
east  south west  west  west 
north north north west  west 
north north west  north west 
east  north north west  north
north north west  north north

Optimal Policy (arrows):
→ ↓ ← ← ←
↑ ↑ ↑ ← ←
↑ ↑ ← ↑ ←
→ ↑ ↑ ← ↑
↑ ↑ ← ↑ ↑
